In [21]:
import collections
import dask
import gcsfs
import h5py
import io
import numba
import numpy as np
import os
import os.path
import pickle
import requests

from dask.distributed import Client
from dask_kubernetes import KubeCluster

In [6]:
cluster = KubeCluster.from_yaml('worker-spec.yml')
#cluster.scale_up(10)  # specify number of nodes explicitly

#cluster.adapt(minimum=1, maximum=100)  # or dynamically scale based on current workload

#client = Client()
#client
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.36.0.20:46277
distributed.scheduler - INFO -   dashboard at:                     :8787


In [ ]:
if not os.path.exists('../../.gcs_tokens'):
    # Get a token
    gcsfs.GCSFileSystem(project='neuron-jungle', token='browser')

In [8]:
with open('../.gcs_tokens', 'rb') as f:
    credentials = pickle.load(f)
credentials = credentials[list(credentials.keys())[0]]
fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
#fs.ls('l4dense/neuron')

['l4dense/dendrites.hdf5',
 'l4dense/segmentation-volume/',
 'l4dense/segments.hdf5']

# Create a map from segment id to neuron id

In [14]:
def locally_cache(filename, credentials):
    fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
    with fs.open(f'l4dense/{filename}', 'rb') as f:
        data = f.read()
    with open(f'../cache/{filename}', 'wb') as f:
        f.write(data)
    return len(data)

locally_cache('dendrites.hdf5', credentials)

263193540

In [16]:
dendrites['dendrites']['agglomerate']

<HDF5 group "/dendrites/agglomerate" (11400 members)>

In [19]:
dendrites = h5py.File('../cache/dendrites.hdf5', 'r')

d = collections.defaultdict(lambda: [])
neuronId = np.array(dendrites['dendrites']['neuronId'])
for i, id in enumerate(neuronId):
    if id > 0:
        # Append the dendrite ids to the right slot.
        d[id] += np.array(dendrites['dendrites']['agglomerate'][str(i + 1)]).tolist()
    
neuron_map = {}
for neuron_id, segment_ids in d.items():
    for segment_id in segment_ids:
        neuron_map[segment_id] = neuron_id

In [22]:
@numba.jit(nopython=True)
def remap(data, the_map):
    b = np.zeros_like(data)
    for i in range(len(data)):
        if data[i] in the_map:
            b[i] = the_map[data[i]]
    return b

In [ ]:
# To repaint: map dendrite ids to neuron id (default to 0)


def repaint(filename, credentials, segment_to_neuron):    
    # Create a typed map for segment_to_neuron
    the_map_typed = Dict.empty(key_type=numba.int32, value_type=numba.uint8)
    for k, v in segment_to_neuron.items():
        the_map_typed[k] = v
        
    fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
    with fs.open(f'l4dense/segmentation-volume/{filename}', 'rb') as f:    
        cube = h5py.File(f, 'r')
        
        a = np.zeros((1024, 1024, 1024), dtype=np.uint8)
        
        slice_size = 32
        nslices = int(1024 / slice_size)
        
        for j in range(nslices):
            subd = np.array(cube['data'][(slice_size*j):(slice_size*(j+1)), :, :])
            for i in range(nslices):
                a[i + j*slice_size, :, :] = remap(subd[i, :, :].ravel(), 
                                   the_map_typed).astype(np.uint8).reshape((1024, 1024))
    
    bio = io.BytesIO()
    cube = h5py.File(bio, 'w')
    cube.create_dataset('data', a.shape, compression="gzip", data=a)
    cube.close()

    with fs.open(f'l4dense/neuron-volume/{filename}', 'wb') as f:
        f.write(bio.read())
    return a.max()

changed = repaint("x0y0z0.hdf5", credentials, segment_to_neuron)
changed.compute()